In [1]:
import django
import os
import sys
import shutil

path='../'

source_file_path = f'{path}.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

from dotenv import dotenv_values

_config = dotenv_values(".env")
_username = _config["username"]

session_file_path = f'{path}{_username}.session'

# shutil.copy(session_file_path, destination_file_path)


project_path = path  # Adjust this to your actual project path
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()


In [2]:
from asgiref.sync import sync_to_async
from PostAnalyzer.models import (
    Predict,
)
import warnings
from Strategies.AbsStrategy import AbsStrategy
from Strategies.Strategy1.Strategy import Strategy1
from Strategies.Strategy2.Strategy import Strategy2
from Strategies.Strategy3.Strategy import Strategy3
from Strategies.Strategy4.Strategy import Strategy4
from Strategies.Strategy10.Strategy import Strategy10
from Strategies.Strategy11.Strategy import Strategy11
from Strategies.Strategy12.Strategy import Strategy12

config = dotenv_values(".env")


# Disable all warnings
warnings.filterwarnings("ignore")




In [3]:
async def loadPredicts(channel_name:str):
    return await sync_to_async(
        lambda: list(
           Predict.objects.filter(
            post__channel__channel_id=config[channel_name]  
            )
        .order_by('date')
        .select_related('post', 'status', 'position', 'symbol', 'market')
        )
    )()

In [4]:
from Shared.helpers import print_colored


close_tp_array = [1, 2, 3, 4, 5, 100]

predicts_CHANNEL_RASTAD = await loadPredicts("CHANNEL_RASTAD")
predicts_CHANNEL_FEYZ = await loadPredicts("CHANNEL_FEYZ")
predicts_CHANNEL_ALI_BEY = await loadPredicts("CHANNEL_ALI_BEY")


predicts_arr = [
    {
        'name': 'CHANNEL_RASTAD',
        'predicts': predicts_CHANNEL_RASTAD
    },
    {
        'name': 'CHANNEL_FEYZ',
        'predicts': predicts_CHANNEL_FEYZ
    },
    {
        'name': 'CHANNEL_ALI_BEY',
        'predicts': predicts_CHANNEL_ALI_BEY
    }
]


max_percent_stoploss=8
positionSize=25
trendPeriod=50
showPrint=False

all_strategy_report = [
    # {
    #     'channel_name': '',
    #     'strategy_name': '',
    #     'close_tp': '',
    #     'initial_money': '',
    #     'total_opening_orders': '',
    #     'total_loss': '',
    #     'total_profit': '',
    #     'gross': '',
    #     'total_pending': '',
    #     'my_free_money': '',
    #     'my_total_money': '',
    #     'profit_count': '',
    #     'loss_count': '',
    #     'pending_count': '',
    #     'missed_count': '',
    # }
]

def addToReport(strategy: AbsStrategy, channel:str, close_tp:int):
    all_strategy_report.append(  {
        'channel_name': channel,
        'strategy_name': strategy.strategy_name,
        'close_tp': close_tp,
        'initial_money': strategy.initial_money,
        'total_opening_orders': strategy.total_opening_orders,
        'total_loss': round(strategy.total_loss*100, 2),
        'total_profit': round(strategy.total_profit*100, 2),
        'gross': round((strategy.total_profit+strategy.total_loss )*100, 2),
        'total_pending': strategy.total_pending_money,
        'my_free_money': round(strategy.current_money-strategy.total_pending_money, 2),
        'my_total_money': round(strategy.current_money, 2),
        'profit_count': strategy.profit_count,
        'loss_count': strategy.loss_count,
        'pending_count': strategy.pending_count,
        'missed_count': len(strategy.missed_orders),

    })


for pr in predicts_arr:
    print_colored(f'\n {pr['name']} ', 'orange')
    predicts = pr['predicts']

    for ct in close_tp_array:
        print_colored(f'\n************ strategy_1 close_tp: {ct}*******************', '#ff0')
        my_money = 1000
        strategy1 = Strategy1(money=my_money,timeZone='Asia/Tehran')
        await strategy1.backtest_with_money_strategy_1(predicts=predicts, close_tp=ct, showPrint=  showPrint, positionSize=positionSize)
        strategy1.report() 
        addToReport(strategy1, pr['name'], ct)

        
        # print_colored(f'\n************ strategy_2 close_tp: {ct}*******************', '#f00')
        # my_money = 1000
        # strategy2 = Strategy2(money=my_money,timeZone='Asia/Tehran')
        # await strategy2.backtest_with_money_strategy_2(predicts=predicts, close_tp=ct, showPrint= showPrint, positionSize=positionSize, max_percent_stoploss=max_percent_stoploss)
        # strategy2.report()
        # addToReport(strategy2, pr['name'], ct)
 
        # print_colored(f'\n************ strategy_3 close_tp: {ct}*******************', '#ccc')
        # my_money = 1000
        # strategy3 = Strategy3(money=my_money,timeZone='Asia/Tehran')
        # await strategy3.backtest_with_money_strategy_3(predicts=predicts, close_tp=ct, showPrint= showPrint, positionSize=positionSize, max_percent_stoploss=max_percent_stoploss, effect_stoploss=True)
        # strategy3.report()
        # addToReport(strategy3, pr['name'], ct)
 
        # print_colored(f'\n************ strategy_4 close_tp: {ct}*******************', '#00f')
        # my_money = 1000
        # strategy4 = Strategy4(money=my_money,timeZone='Asia/Tehran')
        # await strategy4.backtest_with_money_strategy_4(predicts=predicts, close_tp=ct, showPrint= showPrint, positionSize=positionSize, max_percent_stoploss=max_percent_stoploss)
        # strategy4.report()
        # addToReport(strategy4, pr['name'], ct)

        # print_colored(f'\n************ strategy_10 close_tp: {ct}*******************', '#ff0fff')
        # my_money = 1000
        # strategy10 = Strategy10(money=my_money,timeZone='Asia/Tehran')
        # await strategy10.backtest_with_money_strategy_10(predicts=predicts, close_tp=ct, showPrint= showPrint, positionSize=positionSize)
        # strategy10.report()
        # addToReport(strategy10, pr['name'], ct)

        # print_colored(f'\n************ strategy_11 close_tp: {ct}*******************', '#0f0')
        # my_money = 1000
        # strategy11 = Strategy11(money=my_money,timeZone='Asia/Tehran')
        # await strategy11.backtest_with_money_strategy_11(predicts=predicts, close_tp=ct, showPrint= showPrint, positionSize=positionSize, trendPeriod=trendPeriod)
        # strategy11.report()
        # addToReport(strategy11, pr['name'], ct)

        # print_colored(f'\n************ strategy_12 close_tp: {ct}*******************', '#0ff')
        # my_money = 1000
        # strategy12 = Strategy12(money=my_money,timeZone='Asia/Tehran')
        # await strategy12.backtest_with_money_strategy_12(predicts=predicts, close_tp=ct, showPrint= showPrint, positionSize=positionSize, trendPeriod=trendPeriod, risk_percent= 0.6)
        # strategy12.report()
        # addToReport(strategy12, pr['name'], ct)

    print_colored(f'\n****************************** finish *********************************', 'orange')
    



In [ ]:
import pandas as pd

df = pd.DataFrame(all_strategy_report)

# df.to_csv(f'test-all-strategies.csv', index=False)
# df.to_json(f'test-all-strategies.json', index=False)
